# Projected-Gradient-Based Attack

* Load the learned relu/kwta/kaf-based models.
* Craft a PGD-attack for each of them and evaluate robustness based on the activation function used.
* Train every model against PGD and evaluate the resulting robustness.
* How does kwta and kaf activation functions affect the robustness of the network wrt traditional functions such as relu?

In [1]:
# Imports
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
from kafnet import Kaf
from kwta import Kwta

## Load CIFAR10 and the Models

In [3]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train/ 255.0, x_test/ 255.0

Using TensorFlow backend.


In [8]:
## QUI IL PROBLEMA DI ACCURATEZZA! Se valuato subito dopo l allenamento infatti il modello presenta un accuracy ~ 0.73
kwta_cnn_tmp = tf.keras.models.load_model('saved_custom_model/kwta_cnn', custom_objects= {'Kwta': Kwta})
kwta_cnn_tmp.evaluate(x = x_test, y = y_test, batch_size=128)

10000/10000 [==============================] - 9s 889us/sample - loss: 1.1458 - accuracy: 0.6391


[1.145782506942749, 0.6391]

In [5]:
relu_cnn = tf.keras.models.load_model('saved_custom_model/relu_cnn')
kwta_cnn = tf.keras.models.load_model('saved_custom_model/kwta_cnn')
kaf_cnn = tf.keras.models.load_model('saved_custom_model/kaf_cnn')

relu_cnn.summary()
kwta_cnn.summary()
kaf_cnn.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_33 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_34 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 16, 16, 64)       

In [6]:
relu_cnn.evaluate(x_test, y_test, batch_size=64)
kwta_cnn.evaluate(x = x_test, y = y_test, batch_size=64)
kaf_cnn.evaluate(x_test, y_test, batch_size=64)

10000/10000 [==============================] - 11s 1ms/sample - loss: 0.6115 - accuracy: 0.8235


[0.6115004356384277, 0.8235]

## Part 3. Crafting a PGD Attack 

In [ ]:
# We use the IBM Adversarial Robustness 360 Toolbox
# !pip install adversarial-robustness-toolbox

In [ ]:
from art.classifiers import TensorFlowV2Classifier
from art.attacks import ProjectedGradientDescent

# Wrap the model in a TensorFlowV2Classifier object
relu_cnn_art = TensorFlowV2Classifier(model=relu_cnn, nb_classes=10, input_shape=(32, 32, 3), loss_object=tf.keras.losses.SparseCategoricalCrossentropy(), clip_values=(0, 1))
kwta_cnn_art = TensorFlowV2Classifier(model=kwta_cnn, nb_classes=10, input_shape=(32, 32, 3), loss_object=tf.keras.losses.SparseCategoricalCrossentropy(), clip_values=(0, 1))
kaf_cnn_art = TensorFlowV2Classifier(model=kaf_cnn, nb_classes=10, input_shape=(32, 32, 3), loss_object=tf.keras.losses.SparseCategoricalCrossentropy(), clip_values=(0, 1))

In [ ]:
pgd_relu = ProjectedGradientDescent(classifier= relu_cnn_art, eps=0.031, eps_step=0.003, max_iter=40, num_random_init=40, norm=np.inf)
pgd_kwta = ProjectedGradientDescent(classifier= kwta_cnn_art, eps=0.031, eps_step=0.003, max_iter=40, num_random_init=40, norm=np.inf)
pgd_kaf = ProjectedGradientDescent(classifier= kaf_cnn_art, eps=0.031, eps_step=0.003, max_iter=40, num_random_init=40, norm=np.inf)

In [ ]:
# Our net is 90% certain of our image being a horse
classes= ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
plt.imshow(x)
print(classes[np.argmax(reluCnn.predict(tf.expand_dims(x, 0)))])
print(reluCnn.predict(tf.expand_dims(x, 0)))

x_adv = pgd.generate(tf.expand_dims(x, 0))
plt.imshow(tf.squeeze(x_adv))

# Now its 99% sure it is a frog instead!
classes= ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
print(classes[np.argmax(reluCnn.predict(tf.expand_dims(tf.squeeze(x_adv), 0)))])
print(reluCnn.predict(tf.expand_dims(tf.squeeze(x_adv), 0)))

In [ ]:
sample_ids = np.random.randint(1000, size=100)
x_test_sample = x_test[sample_ids, :, :, :]

x_test_sample.shape
x_adv_test = pgd.generate(x_test_sample)
np.save('adversarial_pgd_test_100', x_adv_test)